<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2025</font>
</p>

# Tabla de contenidos

1. [*QThreads* y señales](#QThreads-y-señales)
    1. [Explicación y solución](#Explicación-y-solución)
2. [*isAutoRepeat* en *KeyPressEvent* ](#isAutoRepeat-en-KeyPressEvent)
3. [Sonidos en PyQT](#Sonidos-en-PyQT)
    1. [`QMediaPlayer`](#QMediaPlayer)
    2. [`QSoundEffect`](#QSoundEffect) 
5. [Versiones PyQT](#Versiones-PyQT)

## *QThreads* y señales

En los ejemplos anteriores, se muestra el uso de `Thread`, `QThread`, que envían cambios a una ventana siempre mediante **señales**. Las señales no son la única forma de conseguir este comportamiento, pero sí son las **más escalables para generar cambios** de la interfaz gráfica.

A continuación se muestra un ejemplo de `QThread` que modifica directamente la posición de *labels* dentro de una ventana. Se crean 100 *threads*, cada uno con un `QLabel` que tiene un `QPixmap` de diferente color. La posición evoluciona al pasar el tiempo y en cada cambio el *thread* cambia **directamente** la posición de la etiqueta mediante `label.move()`. 

Si ejecutas en tu computador este código verás que funciona, pero luego de decenas o centenas de *threads* creados, el programa **colapsa** o bien no se ve nada moviéndose. Este código se encuentra en el archivo `3-pyqt-miscelaneo/1_sin_signal.py`.

```python
from random import randint
from time import sleep

from PyQt5.QtCore import QThread
from PyQt5.QtGui import QPixmap, QColor
from PyQt5.QtWidgets import QLabel, QWidget, QApplication
import sys


class Cuadrado(QThread):
    identificador = 0

    def __init__(self, label: str, limite_x: int, limite_y: int):
        super().__init__()
        self.id = Cuadrado.identificador
        Cuadrado.identificador += 1

        # guardamos el label
        self.label = label

        # Seteamos la posición inicial y la guardamos para usarla como una property
        self._posicion = [0, 0]
        self.posicion = [randint(0, limite_x), randint(0, limite_y)]

    @property
    def posicion(self) -> list:
        return self._posicion

    # Cada vez que se actualicé la posición,
    # se actualiza la posición de la etiqueta
    @posicion.setter
    def posicion(self, nueva_posicion: list) -> None:
        self._posicion = nueva_posicion
        nuevo_x, nuevo_y = self.posicion
        self.label.move(nuevo_x, nuevo_y)

    def run(self) -> None:
        while True:
            sleep(0.1)
            nuevo_x = self.posicion[0] + randint(-2, 2)
            nuevo_y = self.posicion[1] + randint(-2, 2)
            self.posicion = [nuevo_x, nuevo_y]


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("No uso de señales")
        self.setGeometry(200, 200, 500, 500)

        # Definimos QLabel para el fondo de la ventana
        self.fondo = QLabel(self)
        self.fondo.setStyleSheet("background: orange")
        self.fondo.setGeometry(0, 0, 500, 500)

        self.cuadrados = []
        self.labels = {}

        for _ in range(100):
            self.crear_cuadrado()

        self.show()

    def crear_cuadrado(self) -> None:
        # Creamos el label y se lo pasamos al Cuadrado
        label = QLabel(self)
        label.setGeometry(-50, -50, 50, 50)

        # Creamos un QPixmap de color aleatorio
        pixmap = QPixmap(50, 50)
        pixmap.fill(QColor(randint(20, 200), randint(20, 200), randint(20, 200)))
        label.setPixmap(pixmap)
        label.show()

        nuevo_cuadrado = Cuadrado(label, self.width(), self.height())
        self.cuadrados.append(nuevo_cuadrado)
        nuevo_cuadrado.start()


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

### Explicación y solución

Esto ocurre debido a que hay múltiples *threads* haciendo cambios directos en la interfaz casi simultáneamente, lo que da espacio para potenciales errores de concurrencia, que son clásicos en *threads*.

En cambio, al usar señales se evade de mejor forma este problema ya que se delega al manejo de eventos de PyQt la realización de los cambios en la interfaz. El siguiente código muestra una adaptación del código anterior pero utilizando señales. Al ejecutarlo, verás que se actualizan de mejor forma los Labels en la ventana y no se cae con los 100 *threads* iniciados. Este código se encuentra en el archivo `3-pyqt-miscelaneo/2_con_signal.py`..

```python
from random import randint
from time import sleep

from PyQt5.QtCore import QThread, pyqtSignal
from PyQt5.QtGui import QPixmap, QColor
from PyQt5.QtWidgets import QLabel, QWidget, QApplication
import sys


class Cuadrado(QThread):
    identificador = 0

    def __init__(self, senal_mover: pyqtSignal, limite_x: int, limite_y: int) -> None:
        super().__init__()
        self.id = Cuadrado.identificador
        Cuadrado.identificador += 1

        # guardamos la señal
        self.senal_mover = senal_mover

        # Seteamos la posición inicial y la guardamos para usarla como una property
        self._posicion = [0, 0]
        self.posicion = (randint(0, limite_x), randint(0, limite_y))

    @property
    def posicion(self) -> list:
        return self._posicion

    # Cada vez que se actualice la posición,
    # se actualiza la posición de la etiqueta
    @posicion.setter
    def posicion(self, nueva_posicion: list) -> None:
        self._posicion = nueva_posicion
        nuevo_x, nuevo_y = self.posicion
        self.senal_mover.emit(self.id, nuevo_x, nuevo_y)

    def run(self):
        while True:
            sleep(0.1)
            nuevo_x = self.posicion[0] + randint(-2, 2)
            nuevo_y = self.posicion[1] + randint(-2, 2)
            self.posicion = (nuevo_x, nuevo_y)


class MiVentana(QWidget):
    senal_mover = pyqtSignal(int, int, int)

    def __init__(self) -> None:
        super().__init__()
        self.setWindowTitle("Correcto uso de señales")
        self.setGeometry(200, 200, 500, 500)

        # Definimos QLabel para el fondo de la ventana
        self.fondo = QLabel(self)
        self.fondo.setStyleSheet("background: orange")
        self.fondo.setGeometry(0, 0, 500, 500)

        self.cuadrados = []
        self.labels = {}

        for _ in range(100):
            self.crear_cuadrado()

        self.senal_mover.connect(self.mover)
        self.show()

    def crear_cuadrado(self) -> None:
        # Creamos el label y se lo pasamos al Cuadrado
        label = QLabel(self)
        label.setGeometry(-50, -50, 50, 50)

        # Creamos un QPixmap de color aleatorio
        pixmap = QPixmap(50, 50)
        pixmap.fill(QColor(randint(20, 200), randint(20, 200), randint(20, 200)))
        label.setPixmap(pixmap)
        label.show()

        nuevo_cuadrado = Cuadrado(self.senal_mover, self.width(), self.height())
        self.labels[nuevo_cuadrado.id] = label
        self.cuadrados.append(nuevo_cuadrado)
        nuevo_cuadrado.start()

    def mover(self, id: int, x: int, y: int) -> None:
        self.labels[id].move(x, y)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())

```

La conclusión de este experimento es que, en general, el uso de señales en conjunto a *theading* permite evadir potenciales problemas de concurrencia.

## *isAutoRepeat* en *KeyPressEvent* 

Anteriormente aprendimos del método `keyPressEvent` para detectar cuando una tecla del teclado es presionada. Una situación de interés de este método es cuando mantenemos presionada una tecla por mucho tiempo. Por defecto, cuando ocurre este suceso, el método es llamado múltiples veces, pero hay situaciones donde no queremos que se ejecute múltiples veces el método si es que mantengo presionado. 

Aquí es donde sale al rescate el método `isAutoRepeat()`. Este método retorna un booleano que indica si el evento gatillado fue producto de presionar por primera vez una tecla (`evento.isAutoRepeat() == False`) o es la repetición del evento por mantener presionada la tecla (`evento.isAutoRepeat() == True`).

A continuación veremos un ejemplo donde detectamos y contamos cuántas veces entramos en un `if` cuando presionamos la letra `A` o la `W`. La diferencia es que la letra `A` incluye una verificación de que `evento.isAutoRepeat()` sea `False`. Este código se encuentra en el archivo `3-pyqt-miscelaneo/3_autorepeat.py`.


```python
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QKeyEvent
from PyQt5.QtWidgets import QApplication, QWidget, QLabel


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        # Configuramos los widgets de la interfaz
        self.label_w = QLabel("Presiona la tecla W", self)
        self.label_w_contador = QLabel("Presionada 0 veces", self)
        self.contador_w = 0

        self.label_a = QLabel("Presiona la tecla A", self)
        self.label_a_contador = QLabel("Presionada 0 veces", self)
        self.contador_a = 0

        self.label_w.setGeometry(10, 10, 230, 30)
        self.label_w_contador.setGeometry(10, 40, 230, 30)
        self.label_a.setGeometry(10, 100, 230, 30)
        self.label_a_contador.setGeometry(10, 130, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo isAutoRepeat")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def keyPressEvent(self, evento: QKeyEvent) -> None:
        if evento.key() == Qt.Key.Key_W:
            self.contador_w += 1
            self.label_w_contador.setText(f"Presionada {self.contador_w} veces")

        if evento.key() == Qt.Key.Key_A and not evento.isAutoRepeat():
            self.contador_a += 1
            self.label_a_contador.setText(f"Presionada {self.contador_a} veces")


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

## Sonidos en PyQT

Cuando uno interactúa con una interfaz gráfica, no puede faltar su música de fondo o algún sonido cuando ocurre algún evento. Por esto, ahora veremos cómo podemos reproducir archivos de audio desde la interfaz. En esta ocasión, veremos 2 objetos de PyQT para reproducir sonido.

Para ambos casos, vamos a utilizar un objeto de PyQt que nos construirá un _path_ absoluto a partir de un _path_ relativo. Esta acción es necesaria para que PyQt sea capaz de reproducir los sonidos.

**Warning**. Para los que tienen **Windows**, es posible llegar a tener un problema si el _path_ absoluto tiene un largo mayor a 260 caracteres. Esto es por una limitación del propio Sistema Operativo que solo se puede deshabilitar con permisos de administrador. Para evitar este error, se recomienda no tener los códigos de IIC2233 en un directorio profundo del computador, así el _path_ absoluto no será mayor a 260. O bien, reinstalar Python y asegurar de presionar la opción _"Disable path length limit"_.

### `QMediaPlayer`

Este objeto nos permite reproducir archivos `.mp3`. Para configurar correctamente este objeto, se debe:

1. Instanciar el objeto: `self.player = QMediaPlayer(self)`
2. Obtener, **dinámicamente**, el _path_ absoluto al sonido que queremos reproducir a partir de un _path_ relativo: `path = os.path.abspath(join("sounds", "waku-waku.mp3"))`.
3. Definir un objeto tipo `QURL` con el _path_ a nuestro sonido: `file_url = QUrl.fromLocalFile(path)`.
4. Crear un objeto tipo `QMediaContent` que utiliza el objeto tipo `QURL`: `content = QMediaContent(QUrl.fromLocalFile(path))`
5. Entregar el objeto tipo `QMediaContent` al reproductor: `self.player.setMedia(content)`
6. Usar `.play()` para reproducir el sonido.
    
**Importante**: por definición, el formato MP3 utiliza un algoritmo con pérdida para conseguir un menor tamaño de archivo. Esto puede ocasionar que al momento de reproducir un sonido con `QMediaPlayer`, salga el siguiente _warning_ en la consola: _Could not update timestamps for skipped samples_ o _Could not update timestamps for discarded samples._ Para efectos de este curso, no te preocupes por dicho _warning_.
    
    
### `QSoundEffect`

Este objeto nos permite reproducir archivos `.wav`. Para configurar correctamente este objeto, se debe: 

1. Instanciar el objeto: `self.media_player_wav = QSoundEffect(self)`
2. Definir un objeto tipo `QURL` con el _path_ a nuestro sonido: `file_url = QUrl.fromLocalFile(join("sounds", "see-you-again.wav"))`.
3. Entregarle la URL a nuestro reproductor: `self.media_player_wav.setSource(file_url)`
4. Usar `.play()` para reproducir el sonido. También podemos usar `stop()` para detenerlo.

A continuación se muestra un ejemplo donde utilizamos ambos objetos para reproducir 2 sonidos distintos. Este código se encuentra en el archivo `3-pyqt-miscelaneo/4_sonidos.py`. Dado el uso de _paths_ relativos, la terminal **debe** estar posicionada dentro del directorio `scripts/3-pyqt-miscelaneo` para hacer `python3 4_sonidos.py`.

```python
import sys
import os
from PyQt5.QtCore import QUrl
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton
from PyQt5.QtMultimedia import QMediaPlayer, QSoundEffect, QMediaContent
from os.path import join


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()

        # Configuramos los widgets de la interfaz
        self.boton_sorpresa = QPushButton("Wooow", self)
        self.boton_empezar = QPushButton("Empezar música de fondo", self)
        self.boton_parar = QPushButton("Parar música de fondo", self)

        self.boton_sorpresa.setGeometry(10, 10, 230, 30)
        self.boton_empezar.setGeometry(10, 50, 230, 30)
        self.boton_parar.setGeometry(10, 90, 230, 30)

        self.boton_sorpresa.clicked.connect(self.empezar_sonido_sorpresa)
        self.boton_empezar.clicked.connect(self.empezar_musica_fondo)
        self.boton_parar.clicked.connect(self.parar_musica_fondo)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Sonidos")
        self.setGeometry(50, 50, 250, 200)

        # Opción MP3: QMediaPlayer en PyQt5 tiene pequeñas diferencias
        # 1. Necesitas utilizar el path absoluto. Para lograr generarlo correctamente
        #    usaremos os.path.abspath que transforma el relativo a absoluto según cada PC
        # 2. Hay que generar un QMediaContent que recibe nuestro QUrl
        # 3. Se utiliza setMedia para indicar el audio a escuchar
        self.player = QMediaPlayer(self)
        path = os.path.abspath(join("sounds", "waku-waku.mp3"))
        content = QMediaContent(QUrl.fromLocalFile(path))
        self.player.setMedia(content)

        # Opción Wav: QSoundEffect
        self.media_player_wav = QSoundEffect(self)
        self.media_player_wav.setVolume(0.1)  # Opcional
        file_url = QUrl.fromLocalFile(join("sounds", "see-you-again.wav"))
        self.media_player_wav.setSource(file_url)

        # Mostrar ventana
        self.show()

    def empezar_sonido_sorpresa(self) -> None:
        self.player.play()

    def empezar_musica_fondo(self) -> None:
        if not self.media_player_wav.isPlaying():
            self.media_player_wav.play()

    def parar_musica_fondo(self) -> None:
        if self.media_player_wav.isPlaying():
            self.media_player_wav.stop()


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

## Versiones PyQT

Ahora que vimos todo lo relevante de `PyQt5`, es importante mencionar que todo lo enseñado en estos contenidos es pensando en el uso de `PyQt5`, si por eventualidades decides utilizar otra versión de `PyQt`, por ejemplo `PyQt6`, puede que sea necesario ajustar los códigos para que funcionen en dicha versión. No siempre los códigos de una versión son iguales a otra, a veces un código sí funcionará en diferentes versiones, pero otras veces no.


Para ejemplificar esta situación, podemos analizar cómo reproducir un sonido MP3 con PyQt6 y PyQt5:

* Versión `PyQt5`
    ```python
    self.player = QMediaPlayer(self)
    path = os.path.abspath(join("sounds", "waku-waku.mp3"))
    content = QMediaContent(QUrl.fromLocalFile(path))
    self.player.setMedia(content)
    ```
<br>

* Versión `PyQt6`
    ```python
    self.media_player_mp3 = QMediaPlayer(self)
    self.media_player_mp3.setAudioOutput(QAudioOutput(self))
    file_url = QUrl.fromLocalFile(join("sounds", "waku-waku.mp3"))
    self.media_player_mp3.setSource(file_url)
    
    ```

Se puede apreciar que en `PyQt6` usando un método llamado `setAudioOutput` y un objeto llamado `QAudioOutput` en vez de `QMediaContent` que es ocupado en `PyQt5`. Esto ocurre porque justamente de una versión a otra, los creadores de una librería pueden alterar las clases del código, y esto es aplicable a cualquier librería y en cualquier lenguaje de programación. Por lo tanto, si decides ocupar otra versión de una librería en el futuro, mucha precaución de que será necesario iterar los códigos para verificar si funcionan.